In [ ]:
import json
import pandas as pd
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import random
import torch
from torch import nn
from torchtext import data
from torchtext.data import TabularDataset, Example

# make the experiment reproducible
SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
TEXT = data.Field(include_lengths = True)
COURSE = data.LabelField(dtype = torch.long)

In [ ]:
fields = [('course', COURSE), ('text', TEXT)]

In [ ]:
all_data = TabularDataset(
    path = 'data/all_WOS.csv',
    format = 'csv',
    fields = fields,
)
train_data = TabularDataset(
    path = 'data/train_WOS.csv',
    format = 'csv',
    fields = fields,
)
test_data = TabularDataset(
    path = 'data/test_WOS.csv',
    format = 'csv',
    fields = fields,
)

In [ ]:
train_data, valid_data = train_data.split(split_ratio=0.9, random_state = random.seed(SEED))

In [ ]:
TEXT.build_vocab(train_data, max_size = 50000)
COURSE.build_vocab(all_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in COURSE vocabulary: {len(COURSE.vocab)}")

In [ ]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = data.Iterator(
    train_data,
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    train=True,
    device = device)

valid_iterator = data.Iterator(
    valid_data,
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    shuffle=False,
    train=False,
    device = device)

test_iterator = data.Iterator(
    test_data,
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    shuffle=False,
    train=False,
    device = device)

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim,
                          hidden_dim,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.activation = nn.Sigmoid()
    
    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.type(torch.IntTensor))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))

        return self.activation(self.fc(hidden))

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 256
HIDDEN_DIM = 200
OUTPUT_DIM = len(COURSE.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM,
           EMBEDDING_DIM,
           HIDDEN_DIM,
           OUTPUT_DIM,
           N_LAYERS,
           BIDIRECTIONAL,
           DROPOUT,
           PAD_IDX)

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def accuracy(preds, y):
    preds = torch.max(preds, axis=1).indices
    correct = (preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def progress_bar(current, final, acc, loss, thing):
    progress = int(100*(current/final))
    remaining = 100-progress
    clear_output(wait=True)
    
    print(thing)
    print('|', end='')
    print('='*progress, end='')
    print('>', end='')
    print(' '*remaining, end='')
    print('|')
    print("Loss:", loss)
    print("Acc:", acc)
    

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.course)
        acc = accuracy(predictions, batch.course)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        torch.cuda.empty_cache()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            text, text_lengths = batch.text            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.course)
            acc = accuracy(predictions, batch.course)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 500

# initialize best loss as infinity
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/course_prediction.pt')
    
    print(f'Epoch: {epoch+38}')
    print(f'\tTrain Loss: {train_loss:.3f}\t|\tTrain Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f}\t|\tVal. Acc: {valid_acc*100:.2f}%')
    

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

In [ ]:
test_acc